# FloPy VTK Export Demo
 This notebook demonstrates how to use FloPy to export to vtk (.vtu) files. This example will cover:
 
     . basic exporting of information for a model, individual package, or array
     . exporting heads and model output data

In [1]:
import os
import sys
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np


try:
    import flopy
except:
    fpth = os.path.abspath(os.path.join('..', '..'))
    sys.path.append(fpth)
    import flopy
    
from flopy.export import vtk
    
print(sys.version)
print('flopy version: {}'.format(flopy.__version__))

flopy is installed in /Users/jdhughes/Documents/Development/flopy_git/flopy_fork/flopy
3.7.3 | packaged by conda-forge | (default, Jul  1 2019, 14:38:56) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
flopy version: 3.2.13


In [2]:
# load model for examples
nam_file = "freyberg.nam"
model_ws = os.path.join("..", "data", "freyberg_multilayer_transient")
ml = flopy.modflow.Modflow.load(nam_file, model_ws=model_ws, check=False)

In [3]:
# output folder to store the outputs from the notebook
workspace = os.path.join(".", "VTK_EXAMPLE_OUTPUTS")
if not os.path.exists(workspace):
    os.mkdir(workspace)

# Exporting to VTK

 for all export calls a folder is provided and the the fmt flag is set to 'vtk'


# Exporting FloPy arrays to .vtu files

    All array exports have the following kwargs:
    
        . smooth: True creates a smooth surface, default is False
        . point_scalars: True outputs point scalar values as well as cell values, default is False.
                   Outputing point scalars will set smooth to True
        . name: A name can be specified to use for the output filename and array scalar name,
            by default the FloPy array name is used
        . binary: True turns on binary vtk export, deault is False

#### Export model top

In [4]:
# create output folder
ot_arrays_folder = os.path.join(workspace, 'arrays_test')
if not os.path.exists(ot_arrays_folder):
    os.mkdir(ot_arrays_folder)
# export model top
model_top_output = os.path.join(ot_arrays_folder, 'TOP')
ml.dis.top.export(model_top_output, fmt='vtk') 

#### Export model bottoms

In [5]:
# 3D Array export
# export model bottoms
model_bottom_output = os.path.join(ot_arrays_folder, 'BOTM')
ml.dis.botm.export(model_bottom_output, fmt='vtk')

#### Export transient array recharge

In [6]:
# transient 2d array
# export recharge
model_recharge_output = os.path.join(ot_arrays_folder, 'RECH')
ml.rch.rech.export(model_recharge_output, fmt='vtk')

#### Export HK with point scalars


In [7]:
# 3D Array export
# hk export, with points
model_hk_export = os.path.join(ot_arrays_folder, 'HK')
ml.upw.hk.export(model_hk_export, smooth=True, fmt='vtk', name='HK', point_scalars=True)

# Package export to .vtu files

    Package export has the following kwargs:
    
        . smooth: True creates a smooth surface, default is False
        . point_scalars: True outputs point scalar values as well as cell values, default is False.
                   Outputing point scalars will set smooth to True
        . binary: True turns on binary vtk export, default is False

#### Export dis and upw package

In [8]:
# package export
# set up package export folder
package_output = os.path.join(workspace, 'package_output_test')
if not os.path.exists(package_output):
    os.mkdir(package_output)
# export dis
dis_output = os.path.join(package_output, 'DIS')
ml.dis.export(dis_output,  fmt='vtk')
#export upw with point scalars
upw_output = os.path.join(package_output, 'UPW')
ml.upw.export(upw_output,  fmt='vtk', point_scalars=True)

# Model export to .vtu files

    Package export has the following kwargs:
    
    . package_names: list of package names to export
    . smooth: True creates a smooth surface, default is False
    . point_scalars: True outputs point scalar values as well as cell values, default is False.
               Outputing point scalars will set smooth to True
    . binary: True turns on binary vtk export, default is False

#### Export model as binary .vtu files

In [9]:

model_output = os.path.join(workspace, 'model_output_test')
ml.export(model_output, fmt='vtk', binary=True)


'./VTK_EXAMPLE_OUTPUTS/model_output_test'

# Export heads

    vtk.export_heads(model, headsfile, output_folder)

    Heads export has the following arguments:

    . nanval: No data value default is -1e20
    . kstpkper: A tuple containing the time step and stress period (kstp, kper) or
        list of (kstp, kper) tuples. The kstp and kper values are zero based.
    . smooth: True creates a smooth surface, default is False
    . point_scalars: True outputs point scalar values as well as cell values, default is False.
               Outputing point scalars will set smooth to True
    . binary: True turns on binary vtk export, default is False

### Export heads to binary .vtu files

#### Exports a .pvd file that can be loaded into Paraview to view heads as time series

In [10]:
# export binary heads
heads_file = os.path.join(model_ws, 'freyberg.hds')
heads_output_folder = os.path.join(workspace, 'heads_output_test')
vtk.export_heads(ml, heads_file, heads_output_folder, binary=True, nanval=-999.99)

### Export heads to binary .vtu files with point head scalars

In [11]:
# export heads with parameters
heads_file = os.path.join(model_ws, 'freyberg.hds')
heads_output_folder = os.path.join(workspace, 'heads_output_test_parameters')
# export heads for time step 1, stress periods 1, 50, 100, 1000
vtk.export_heads(ml, heads_file, heads_output_folder, kstpkper=[(0,0), (0, 49), (0, 99), (0, 999)],
                 point_scalars=True, nanval=-999.99)

# Export output cell by cell file to .vtu

    vtk.export_heads(model, cellbycellfile, outputfolder)

        Heads export has the following arguments:

        . precision: Default is single
        . nanval: No data value default is -1e20
        . kstpkper: A tuple containing the time step and stress period (kstp, kper) or
            list of (kstp, kper) tuples. The kstp and kper values are zero based.
        . text: The text identifier for the record. Can be a str or list of strings.
        . smooth: True creates a smooth surface, default is False
        . point_scalars: True outputs point scalar values as well as cell values, default is False.
                   Outputing point scalars will set smooth to True
        . binary: True turns on binary vtk export, default is False

In [12]:
cbc_file = os.path.join(model_ws, 'freyberg.cbc')
cbc_output_folder = os.path.join(workspace, 'cbc_output_test_parameters')
vtk.export_cbc(ml, cbc_file, cbc_output_folder, kstpkper=[(0, 0), (0, 9), (0, 10), (0, 11)],
               text=['CONSTANT HEAD', 'STORAGE'], point_scalars=True, binary=True)